In [3]:
import shutil
import requests
import numpy as np
import os
from tqdm import tqdm

folder = 'sekinchan'
os.makedirs(folder, exist_ok=True)

In [7]:
# https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames#Tile_numbers_to_lon..2Flat.

import math

def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
    return (xtile, ytile)

def num2deg(xtile, ytile, zoom):
    n = 2.0 ** zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = math.degrees(lat_rad)
    return (lat_deg, lon_deg)

def download_img(lat, lon, zoom = 14, output_name = 'img.png'):
    x, y = deg2num(lat, lon, zoom)
    url = f'https://api.mapbox.com/styles/v1/mapbox/satellite-v9/tiles/{zoom}/{x}/{y}@2x?title=true&access_token=pk.eyJ1IjoibWFwYm94IiwiYSI6ImNpejY4NXVycTA2emYycXBndHRqcmZ3N3gifQ.rJcFIG214AriISLbB6B5aw'
    response = requests.get(url, stream=True)
    with open(output_name, 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)

In [5]:
# download_img(3.692577, 100.999845)

In [9]:
# top left, top right, bottom left, bottom right
boundaries = [[(3.889538, 100.665019), (3.895019, 101.319144), (3.294703, 100.818932), (3.363252, 101.525275)],
           [(4.128441, 101.081405), (4.155835, 101.455190), (3.681789, 100.957727), (3.679048, 101.408467)]]

resolution = 0.01
zoom = 15

for no, boundary in enumerate(boundaries):
    uniques = {}
    boundary = np.array(boundary)
    latmin = np.min(boundary[:,0])
    lonmin = np.min(boundary[:,1])
    latmax = np.max(boundary[:,0])
    lonmax = np.max(boundary[:,1])
    
    lat_arange = np.arange(latmin, latmax, resolution)
    lon_arange = np.arange(lonmin, lonmax, resolution)
    x, y = np.meshgrid(lat_arange, lon_arange)
    x = np.round(x, 10)
    y = np.round(y, 10)
    x = np.reshape(x, (-1, 1))
    y = np.reshape(y, (-1, 1))
    concated = np.concatenate([x, y], axis = 1)
    for c in concated:
        d = deg2num(c[0], c[1], zoom = zoom)
        uniques[str(d)] = (c[0], c[1])
        
    
    for no_, v in tqdm(enumerate(list(uniques.values()))):
        output_name = os.path.join(folder, f'{no}-{no_}.png')
        download_img(v[0], v[1], output_name = output_name, zoom = zoom)

4400it [49:30,  1.48it/s]
1935it [15:02,  2.14it/s]


In [10]:
!du -hs sekinchan

1.1G	sekinchan


In [11]:
!tar -zcf sekinchan-raw.tar.gz sekinchan